# VideoLingo Colab (批处理 / 无 WebUI)

本 Notebook 用于在 Colab 的 T4 GPU 上按步骤运行 VideoLingo 的批处理模式（不使用 Streamlit WebUI）。
输出结果会保存在 `batch/output/`。

准备：LLM/ASR/TTS 的 API Key（若使用云端服务）。


## 0. 选择 T4 GPU
在 Colab 菜单：`Runtime -> Change runtime type -> Hardware accelerator` 选择 **T4 GPU**。


In [ ]:
!nvidia-smi


## 1. 克隆仓库


In [ ]:
!git clone https://github.com/Huanshere/VideoLingo.git
%cd VideoLingo


## 2. 安装依赖（约 5-10 分钟）
包括系统依赖（ffmpeg）与 Python 依赖。


In [ ]:
!apt-get update -y
!apt-get install -y ffmpeg

!pip install -U pip
!pip install torch==2.0.0 torchaudio==2.0.0 --index-url https://download.pytorch.org/whl/cu118
!pip install -r requirements.txt


## 3. 配置 `config.yaml`
只需要修改下面代码块里的变量。

- 如果不做配音，把任务表里的 `Dubbing` 设为 `0` 即可，TTS 配置可以忽略。
- 如果使用云端 WhisperX，把 `WHISPER_RUNTIME` 设为 `"cloud"` 并填写 `WHISPER_302_API_KEY`。


In [ ]:
from pathlib import Path
from ruamel.yaml import YAML

# ====== 需要你填写的配置 ======
LLM_API_KEY = "YOUR_LLM_API_KEY"
LLM_BASE_URL = "https://yunwu.ai"
LLM_MODEL = "gpt-4.1-2025-04-14"
LLM_SUPPORT_JSON = False

TARGET_LANGUAGE = "简体中文"
SOURCE_LANGUAGE = "en"
WHISPER_RUNTIME = "local"  # "local" 或 "cloud"
WHISPER_302_API_KEY = "YOUR_302_API_KEY"

USE_DEMUCS = True
BURN_SUBTITLES = True
YTB_RESOLUTION = "1080"  # 360/1080/best

# 配音（可选）
TTS_METHOD = "edge_tts"  # azure_tts/openai_tts/fish_tts/gpt_sovits 等
EDGE_VOICE = "zh-CN-XiaoxiaoNeural"
# ============================

yaml = YAML()
cfg_path = Path("config.yaml")
with cfg_path.open("r", encoding="utf-8") as f:
    cfg = yaml.load(f)

cfg["api"]["key"] = LLM_API_KEY
cfg["api"]["base_url"] = LLM_BASE_URL
cfg["api"]["model"] = LLM_MODEL
cfg["api"]["llm_support_json"] = LLM_SUPPORT_JSON

cfg["target_language"] = TARGET_LANGUAGE
cfg["whisper"]["language"] = SOURCE_LANGUAGE
cfg["whisper"]["detected_language"] = SOURCE_LANGUAGE
cfg["whisper"]["runtime"] = WHISPER_RUNTIME
cfg["whisper"]["whisperX_302_api_key"] = WHISPER_302_API_KEY

cfg["demucs"] = USE_DEMUCS
cfg["burn_subtitles"] = BURN_SUBTITLES
cfg["ytb_resolution"] = YTB_RESOLUTION

cfg["tts_method"] = TTS_METHOD
cfg["edge_tts"]["voice"] = EDGE_VOICE

with cfg_path.open("w", encoding="utf-8") as f:
    yaml.dump(cfg, f)

print("config.yaml 已更新")


## 4. 准备任务

你可以处理本地视频或 YouTube 链接：
- 本地视频：上传到 `batch/input/`，任务表里填写文件名（不含路径）。
- YouTube：任务表里直接填写链接。

大文件建议先挂载 Google Drive，再复制到 `batch/input/`。


In [ ]:
from google.colab import files
import os
import shutil

os.makedirs("batch/input", exist_ok=True)
uploaded = files.upload()
for name in uploaded:
    shutil.move(name, f"batch/input/{name}")

print("Uploaded:", list(uploaded.keys()))


如果使用 Google Drive，可运行下面代码挂载，然后手动复制文件到 `batch/input/`：


In [ ]:
from google.colab import drive
drive.mount("/content/drive")


创建/更新任务表 `batch/tasks_setting.xlsx`：


In [ ]:
import pandas as pd

tasks = [
    {
        "Video File": "your_video.mp4",  # 也可以写 YouTube 链接
        "Source Language": "en",
        "Target Language": "简体中文",
        "Dubbing": 0,  # 0=不配音, 1=配音
        "Status": ""
    }
]

df = pd.DataFrame(tasks, columns=["Video File", "Source Language", "Target Language", "Dubbing", "Status"])
df.to_excel("batch/tasks_setting.xlsx", index=False)
df


## 5. 运行批处理


In [ ]:
!PYTHONPATH=. python batch/utils/batch_processor.py


## 6. 导出结果
结果会在 `batch/output/`，可以打包下载。


In [ ]:
!ls -la batch/output
!zip -r videolingo_output.zip batch/output

from google.colab import files
files.download("videolingo_output.zip")
